In \[74\]:

    import pygame
    import random
    import sys
    import time

    # Initialize Pygame
    pygame.init()

    # Game Constants
    WIDTH, HEIGHT = 800, 600
    BALL_SPEED = [4, -4]
    PADDLE_SPEED = 8
    BRICK_ROWS, BRICK_COLS = 5, 8
    BRICK_WIDTH, BRICK_HEIGHT = WIDTH // BRICK_COLS, 30
    CHARM_DROP_RATE = 0.3  # 30% chance to drop a charm

    # Colors
    WHITE = (255, 255, 255)
    BLUE = (0, 0, 255)
    RED = (255, 0, 0)
    GREEN = (0, 255, 0)
    ORANGE = (255, 165, 0)
    BLACK = (0, 0, 0)
    PURPLE = (160, 32, 240)  # Charm Color
    YELLOW = (255, 255, 0)  # Powerup Color (for wider paddle)
    GREEN_POWERUP_COLOR = (0, 255, 0)  # Green Powerup Color (for brick color change)

    # Screen Setup
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Brick Breaker")

    # Score Initialization
    score = 0
    font = pygame.font.Font(None, 36)

    # Paddle and Ball Setup
    paddle = pygame.Rect(WIDTH // 2 - 60, HEIGHT - 20, 120, 10)
    balls = [pygame.Rect(WIDTH // 2 - 10, HEIGHT // 2, 20, 20)]
    ball_speeds = [[4, -4]]

    # Bricks with hit points
    bricks = []
    charms = []
    powerups = []  # List to store powerup objects
    wider_paddle_time = 0  # Track the time for the wider paddle powerup
    green_brick_time = 0  # Track the time for the green brick powerup

    for r in range(BRICK_ROWS):
        for c in range(BRICK_COLS):
            x, y = c * BRICK_WIDTH, r * BRICK_HEIGHT
            rect = pygame.Rect(x, y, BRICK_WIDTH - 2, BRICK_HEIGHT - 2)
            health = random.choice([1, 2])  # Some bricks need 2 hits
            bricks.append((rect, health))

    # Start Button
    button_rect = pygame.Rect(WIDTH // 2 - 50, HEIGHT // 2 - 25, 100, 50)

    # Draw Function
    def draw():
        screen.fill(WHITE)
        pygame.draw.rect(screen, BLUE, paddle)
        
        # Draw Balls
        for ball in balls:
            pygame.draw.ellipse(screen, RED, ball)

        # Draw Bricks
        for brick, health in bricks:
            color = GREEN if health == 1 else ORANGE
            if time.time() - green_brick_time < 10:  # Green brick effect for 10 seconds
                color = GREEN  # Change color to green for the powerup
            pygame.draw.rect(screen, color, brick)

        # Draw Powerups
        for powerup in powerups:
            pygame.draw.ellipse(screen, YELLOW, powerup)  # For paddle powerup
            pygame.draw.ellipse(screen, GREEN_POWERUP_COLOR, powerup)  # For green brick powerup

        # Draw Score
        score_text = font.render(f"Score: {score}", True, BLACK)
        screen.blit(score_text, (10, 10))

        pygame.display.flip()

    # Draw Retry Button
    def draw_retry_screen():
        screen.fill(WHITE)
        pygame.draw.rect(screen, BLACK, button_rect)
        text = font.render("Retry", True, WHITE)
        screen.blit(text, (button_rect.x + 25, button_rect.y + 10))
        pygame.display.flip()

    # Game Restart Logic
    def restart_game():
        global score, bricks, balls, ball_speeds, charms, paddle, powerups, wider_paddle_time, green_brick_time

        score = 0
        bricks = []
        charms = []
        powerups = []
        balls = [pygame.Rect(WIDTH // 2 - 10, HEIGHT // 2, 20, 20)]
        ball_speeds = [[4, -4]]
        paddle = pygame.Rect(WIDTH // 2 - 60, HEIGHT - 20, 120, 10)
        wider_paddle_time = 0
        green_brick_time = 0
        
        for r in range(BRICK_ROWS):
            for c in range(BRICK_COLS):
                x, y = c * BRICK_WIDTH, r * BRICK_HEIGHT
                rect = pygame.Rect(x, y, BRICK_WIDTH - 2, BRICK_HEIGHT - 2)
                health = random.choice([1, 2])  # Some bricks need 2 hits
                bricks.append((rect, health))

    # Main Game Loop
    running = True
    waiting = False

    while running:
        pygame.time.delay(16)

        if len(balls) == 0:  # All balls have fallen off the screen
            waiting = True  # Pause the game
            draw_retry_screen()  # Show retry button
            final_score_text = font.render(f"Game Over! Final Score: {score}", True, BLACK)
            screen.blit(final_score_text, (WIDTH // 2 - 100, HEIGHT // 2 + 60))  # Show final score
            pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if event.type == pygame.MOUSEBUTTONDOWN and waiting:
                if button_rect.collidepoint(event.pos):
                    restart_game()  # Restart the game when clicking retry
                    waiting = False  # Exit waiting state

        if not waiting:
            # Paddle Movement
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT] and paddle.left > 0:
                paddle.x -= PADDLE_SPEED
            if keys[pygame.K_RIGHT] and paddle.right < WIDTH:
                paddle.x += PADDLE_SPEED

            # Apply powerups effects
            if time.time() - wider_paddle_time < 10:  # Wider paddle effect for 10 seconds
                paddle.width = 160  # Make paddle wider
            else:
                paddle.width = 120  # Normal paddle size

            # Ball Movement
            for i in range(len(balls)):
                balls[i].x += ball_speeds[i][0]
                balls[i].y += ball_speeds[i][1]

                # Ball Collision with walls
                if balls[i].left <= 0 or balls[i].right >= WIDTH:
                    ball_speeds[i][0] = -ball_speeds[i][0]
                if balls[i].top <= 0:
                    ball_speeds[i][1] = -ball_speeds[i][1]
                if balls[i].colliderect(paddle):
                    ball_speeds[i][1] = -ball_speeds[i][1]

                # Brick Collision
                for j, (brick, health) in enumerate(bricks):
                    if balls[i].colliderect(brick):
                        bricks[j] = (brick, health - 1)
                        ball_speeds[i][1] = -ball_speeds[i][1]
                        score += 10  # Increase score when a brick is hit
                        if bricks[j][1] == 0:  # Brick destroyed
                            if random.random() < CHARM_DROP_RATE:
                                charms.append(pygame.Rect(brick.x + BRICK_WIDTH // 4, brick.y, 20, 20))
                            if random.random() < 0.2:  # 20% chance for powerup drop
                                powerups.append(pygame.Rect(brick.x + BRICK_WIDTH // 4, brick.y, 20, 20))
                            bricks.pop(j)
                        break

                # Remove ball if it falls off the screen
                if balls[i].bottom >= HEIGHT:
                    balls.pop(i)
                    ball_speeds.pop(i)
                    break

            # Powerup Falling
            for powerup in powerups[:]:
                powerup.y += 3
                if powerup.colliderect(paddle):
                    # Activate the green brick powerup
                    green_brick_time = time.time()  # Set the green brick powerup time
                    powerups.remove(powerup)
                elif powerup.y > HEIGHT:
                    powerups.remove(powerup)

            # Drawing
            draw()

    pygame.quit()

In \[ \]: